## -=Подготовка к запуску=-

### Установка пакетов

В случае, если пакеты через requirements.тхт установлены не были, блок ниже установит все необходимые пакеты для работы данного блокнота.

In [ ]:
%pip install ultralytics==8.0.198
%pip install tifffile==2023.9.26
%pip install Pillow==10.0.1
%pip install numpy==1.24.3
%pip install opencv-python==4.6.0.66

### Импорт библиотек

In [ ]:
import cv2
import numpy as np
from PIL import Image
import tifffile
from ultralytics import YOLO

Импорт функций из detectors.py, в данном файле находится пара вспомогательных функций, детектор ансамбля и его система голосования.

In [ ]:
import detectors

## -=Загрузка моделей=-

Всего есть два ансамбля по 3 модели:
1. Для обработки RGB изображений
2. Для обработки tiff изображений

Оба ансамбля были проверены, и имеют примерно одинаковую точность и предсказания.
Обучение модели одновременно на детекцию в tiff и RGB снижает её общую точность.

In [ ]:
#Ансамбль РГБ
model1 = YOLO("models/YOLOv8/Model_type1.pt")
model2 = YOLO("models/YOLOv8/Model_type2.pt")
model3 = YOLO("models/YOLOv8/Model_type3.pt")
models_RGB = [model1, model2, model3]

#Ансамбль tiff
model1_t = YOLO("models/YOLOv8/Model_type1_t.pt")
model2_t = YOLO("models/YOLOv8/Model_type2_t.pt")
model3_t = YOLO("models/YOLOv8/Model_type3_t.pt")
models_tiff = [model1_t, model2_t, model3_t]

## -=Детекция=-

### Детекция на изображениях RGB

Загрузка изображения

In [ ]:
image = Image.open("examples/test0.png")
display(image)

Детекция

In [ ]:
image, results = detectors.ensemble_detect(models_RGB, image)
display(image)
print(f"Дерево:{results[0]}")
print(f"Стекло:{results[1]}")
print(f"Пластик:{results[2]}")
print(f"Металл:{results[3]}")

### Детекция на изображениях tiff

Выбор длины волны из 400, 430, 460, 490, 520, 550, 580, 610, 640, 670, 700

От 0 до 10 соответственно

In [ ]:
wave_length = 0

Загрузка изображения

In [ ]:
image = tifffile.imread('examples/0000.tif')
channel = image[wave_length, :, :]
channel = (channel * 255).astype(np.uint8)
channel_image = Image.fromarray(channel)       
display(channel_image)

Детекция

In [ ]:
image, results = detectors.ensemble_detect(models_tiff, channel_image)
display(image)
print(f"Дерево:{results[0]}")
print(f"Стекло:{results[1]}")
print(f"Пластик:{results[2]}")
print(f"Металл:{results[3]}")

### Детекция на видео и подсчёт общего количества уникальных объектов

Загрузка видео

In [ ]:
video_capture = cv2.VideoCapture('examples/output0.mp4')

Детекция

Детекция уникальных объектов происходит "в лоб": каждые 25 кадров берётся фрейм, результаты детекции подсчитываются и суммируются, да, звучит просто и наивно, но результаты сравнительно неплохие, особенно относительно треко-детекций которых мы находили с открытым исходным кодом.

К сожалению, показ самого видеопотока пока не предусмотрен

In [ ]:
res = []
counter = 0
while True:
    ret, frame = video_capture.read()
    if not ret:
        break  
             
    counter += 1    
    
    processed_frame, results = detectors.ensemble_detect(models_RGB, frame)
    if counter % 25 == 0:   
        res.append(results)
    sums = [sum(x) for x in zip(*res)]
        
print(f"Дерево:{sums[0]}")
print(f"Стекло:{sums[1]}")
print(f"Пластик:{sums[2]}")
print(f"Металл:{sums[3]}")

### Детекция на потоке с камеры устройства

Блок ниже запустит захват с устройства с номеном 0, после чего откроется новое окно, в котором будет показано изображение с потока с детекцией.
Задержка достаточно большая, эт освязано с тем, что в данном случае три модели отрабатывают последовательно, в теории, процесс можно ускорить, если использовать модели в многопоточном режиме.

Чтобы закрыть окно, необходимо нажать клавишу "ESC"

In [ ]:
vid = cv2.VideoCapture(0) 
  
while(True): 
    ret, frame = vid.read() 
    image, results = detectors.ensemble_detect(models_RGB, frame)
    new_image = np.array(image.convert('RGBA'), dtype=np.uint8)
    cv2.imshow('frame', new_image) 
      
    #"ESC" чтобы закрыть окно
    if cv2.waitKey(1) & 0xFF == 27:
        break
   
vid.release() 
cv2.destroyAllWindows() 

В данном блоке демонстрируется скорость работы с одной моделью

In [ ]:
vid = cv2.VideoCapture(0) 
  
while(True): 
    ret, frame = vid.read() 
    #frame = np.array(image.convert('RGB'), dtype=np.uint8)
    image, results = detectors.ensemble_detect([model1], frame)
    new_image = np.array(image.convert('RGBA'), dtype=np.uint8)
    cv2.imshow('frame', new_image) 
      
    #"ESC" чтобы закрыть окно
    if cv2.waitKey(1) & 0xFF == 27:
        break
   
vid.release() 
cv2.destroyAllWindows() 